<a href="https://colab.research.google.com/github/ragavkish/e-com_transaction/blob/main/Ragavkishore_DM_Clustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##**Import Libraries**

In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.metrics import davies_bouldin_score, silhouette_score
import matplotlib.pyplot as plt
import seaborn as sns

##**Load Datasets**

In [3]:
customers = pd.read_csv('/content/datasets/Customers.csv')
products = pd.read_csv('/content/datasets/Products.csv')
transactions = pd.read_csv('/content/datasets/Transactions.csv')

##**Merge datasets**

In [4]:
data = transactions.merge(customers, on='CustomerID').merge(products, on='ProductID')

##**Feature Engineering**

##**Encode categorical variables**

##**Normalize features**

##**Clustering**

##**Metrics**

##**PCA for visualization**

##**Visualize clusters**